# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [82]:
import pandas as pd
import os
import platform
import re
import json
import yaml
import shutil
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [83]:
TEST_NAME = "t5-large"

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
OUTPUT_PATH = PRE_PATH + "output/" # 모델의 출력 값
CHECKPOINT_PATH = PRE_PATH + "checkpoint/" # 모델의 최종 출력 값
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
LOG_PATH = PRE_PATH + "log/"
TRAIN_PATH = DATA_PATH + "train_new2.csv"
VALID_PATH = DATA_PATH + "dev_new2.csv"
TEST_PATH = DATA_PATH + "test.csv"
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

# 허깅페이스 관련
# HUGGING_MODEL = "digit82/kobart-summarization"
# HUGGING_MODEL = "EbanLee/kobart-summary-v3"
# HUGGING_MODEL = "gogamza/kobart-base-v2"
# HUGGING_MODEL = "gogamza/kobart-summarization"
# HUGGING_MODEL = "alaggung/bart-r3f"
# HUGGING_MODEL = "NLPBada/kobart-chat-persona-extraction-v2"
HUGGING_MODEL = "lcw99/t5-large-korean-text-summary"

# 완디비 관련
WANDB_ENTITY = 'helpot'
WANDB_PROJECT_NAME = 'bootcamp-upstage-nlp'
WANDB_RUN_NAME = TEST_NAME
WANDB_LOG = "end" # end or checkpoint
WANDB_WATCH = "false" # true or false


In [84]:
tokenizer = AutoTokenizer.from_pretrained(HUGGING_MODEL)

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [85]:
loaded_config = {
    "general": {
        "data_path": DATA_PATH,
        "output_dir": OUTPUT_PATH,
        "model_name": HUGGING_MODEL,
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 200,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#PassportNumber#', '#Person#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#SSN#']
    },
    "wandb": {
        "entity": WANDB_ENTITY,
        "project": WANDB_PROJECT_NAME,
        "name": WANDB_RUN_NAME
    },
    "inference": {
        "ckt_path": CHECKPOINT_PATH,
        "result_path": PREDICTION_PATH,
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [86]:
# 데이터 전처리를 위한 클래스: 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self, prefix: str) -> None:
        self.prefix = prefix  # T5 모델에서 사용할 태스크 프리픽스 (예: "summarize: ")

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train=True):
        df = pd.read_csv(file_path)
        if is_train:
            train_df = df[['fname', 'dialogue', 'summary']]
            return train_df
        else:
            test_df = df[['fname', 'dialogue']]
            return test_df

    # T5 모델의 입력 형태에 맞게 전처리를 진행합니다.
    def make_input(self, dataset, is_test=False):
        if is_test:
            # 테스트 데이터셋의 경우, 입력 텍스트에 태스크 프리픽스를 추가합니다.
            encoder_input = dataset['dialogue'].apply(lambda x: self.prefix + str(x))
            decoder_input = [""] * len(dataset['dialogue'])  # T5 모델에서는 테스트 시 디코더 입력이 필요하지 않습니다.
            return encoder_input.tolist(), decoder_input
        else:
            # 학습 데이터셋의 경우, 입력 텍스트에 태스크 프리픽스를 추가하고, 출력 텍스트로 학습합니다.
            encoder_input = dataset['dialogue'].apply(lambda x: self.prefix + str(x))
            decoder_input = dataset['summary'].apply(lambda x: str(x))  # T5 모델에서는 디코더 입력을 별도로 준비하지 않습니다.
            decoder_output = dataset['summary'].apply(lambda x: str(x))  # 목표 요약 문장을 디코더 출력으로 사용합니다.
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [87]:
from torch.utils.data import Dataset

# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, labels, length):
        self.encoder_input = encoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        # 디코더의 labels는 요약문으로 설정
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return self.length

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, labels, length):
        self.encoder_input = encoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        
        # 디코더의 labels는 요약문으로 설정
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return self.length

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, length):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.length = length

    def __getitem__(self, idx):
        # 인코더 입력: input_ids와 attention_mask
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.length


## 4. 모델 추론하기

In [88]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "/kkh/output/checkpoint-94752/"

- test data를 사용하여 모델의 성능을 확인합니다.

In [89]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):
    # 테스트 데이터 경로 설정
    test_file_path = os.path.join(config['general']['data_path'], 'test.csv')

    # 테스트 데이터셋 준비
    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']

    print('-' * 150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-' * 150)

    # 인코더 입력 준비 (T5 모델에서는 디코더 입력을 따로 준비하지 않음)
    encoder_input_test, _ = preprocessor.make_input(test_data, is_test=True)
    print('-' * 10, 'Load data complete', '-' * 10,)

    # 인코더 입력을 토크나이저로 토큰화합니다.
    test_tokenized_encoder_inputs = tokenizer(
        encoder_input_test,
        return_tensors="pt",
        padding=True,
        add_special_tokens=True,
        truncation=True,
        max_length=config['tokenizer']['encoder_max_len'],
        return_token_type_ids=False,
    )

    # 테스트 데이터셋 생성
    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-' * 10, 'Make dataset complete', '-' * 10,)

    return test_data, test_encoder_inputs_dataset

In [90]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    # 모델 이름과 체크포인트 경로 설정
    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)

    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 스페셜 토큰 추가
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    # T5 모델 로드
    generate_model = T5ForConditionalGeneration.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    
    # 모델을 지정한 장치로 이동
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model, tokenizer


In [91]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader

# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    # 사용할 장치를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 모델과 토크나이저를 로드합니다.
    generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    # 데이터 경로와 전처리 객체를 정의합니다.
    data_path = config['general']['data_path']
    preprocessor = Preprocess(prefix=config['tokenizer'].get('prefix', 'summarize: '))

    # 테스트 데이터셋을 준비합니다.
    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []

    # 추론을 수행합니다.
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            input_ids = item['input_ids'].to(device)
            
            generated_ids = generate_model.generate(
                input_ids=input_ids,
                no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                early_stopping=config['inference']['early_stopping'],
                max_length=config['inference']['generate_max_length'],
                num_beams=config['inference']['num_beams'],
            )
            
            # 요약문 디코딩 및 저장
            for ids in generated_ids:
                # 스페셜 토큰을 유지하기 위해 skip_special_tokens=False로 설정합니다.
                result = tokenizer.decode(ids, skip_special_tokens=False, clean_up_tokenization_spaces=True)

                # 불필요한 스페셜 토큰 제거
                result = result.replace('<pad>', '').replace('</s>', '').strip()
                summary.append(result)

    # 결과를 데이터프레임으로 변환하고 파일로 저장
    output = pd.DataFrame({"fname": test_data['fname'], "summary": summary})
    output.to_csv(os.path.join(SUBMIT_PATH), index=False)

    return output


In [92]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.3.1+cu121
---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 

100%|██████████| 16/16 [00:36<00:00,  2.28s/it]


In [93]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,더슨 씨는 #Person1# 에게 사무실 통신이 이메일 통신과 공식 메모로 제한...
1,test_1,#Person1# 은 교통 체증에 걸렸다. #Person2# 는 #Person1#...
2,test_2,케이트는 마샤와 히어로가 이혼하려고 한다고 #Person1# 에게 말한다. 그녀...
3,test_3,브라이언은 파티에서 #Person1# 의 목걸이 드레스를 입고 춤을 추고 있다....
4,test_4,#Person1# 과 #Person2# 는 올림픽 공원에 대해 이야기하고 있습니다...
...,...,...
494,test_495,찰리는 아빠를 데리러 가야하기 때문에 새 게임에 대해 잭에게 이야기한다. 잭은 ...
495,test_496,#Person2# 는 #Person1# 에게 #Person2# 가 컨트리 음악에 ...
496,test_497,"앨리스는 #Person1# 에게 기계를 어떻게 사용하는지 알려주고, 기계가 너무..."
497,test_498,스티브와 매튜는 계약이 다음 달에 끝나기 때문에 새로운 집을 찾고 있다. 그들은...
